In [ ]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.signal import find_peaks
from scipy import stats
from patsy import dmatrices, dmatrix
import statsmodels.api as sm
import glob
import os
import bioframe
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cooler
from cooltools import snipping

In [ ]:
plt.rcParams.update({'font.size': 24})
chromsizes=bioframe.fetch_chromsizes('mm10')
chromosomes = list(chromsizes.index)
binsize=5000
flank = 300000
supports = [(chrom, 0, chromsizes[chrom]) for chrom in chromosomes]
binsDf=pd.read_csv('../data/genomeInfo/mm10bin5kb.bed',sep='\t',header=None,names=['chrom','start','end']) #bin file can be made with cooler makebins, binsize = 5000

signalFiles=glob.glob('../data/binnedBedgraphs/*')
signalDf=binsDf.copy()
for signalFile in signalFiles:
    signalDf=signalDf.merge(pd.read_csv(signalFile,sep="\t", header=None,names=['chrom','start','end',os.path.basename(signalFile).split('.')[0]]),how='left',on=['chrom','start','end'])
    signalDf.drop_duplicates(subset=['chrom','start','end'],inplace=True)
varList=signalDf.columns[3:]
rollSignalDf=signalDf.groupby(['chrom'])[varList].rolling(window=101, center=True, min_periods=1).mean().reset_index(level=0).drop(columns=['chrom'])
signalDf=pd.merge(signalDf,rollSignalDf, left_index=True,right_index=True, suffixes=('', '_roll101mean'))

In [ ]:
def getPeakInds(peakFile,binsize=5000,flank=300000,supports=supports):
    peakDf=pd.read_csv(peakFile,header=None,sep="\t",names=['chrom','start','end','name','score','strand','signalValue','pvalue','qvalue','peak'])
    windows = snipping.make_bin_aligned_windows(
        binsize,
        peakDf['chrom'],
        (peakDf['start'] + peakDf['end'])//2,
        flank_bp=flank)
    windows=windows.merge(pd.DataFrame(supports,columns=['chrom','zero','chromsize']),how='left',on='chrom')
    windows=windows[(windows.end<windows.chromsize) & (windows.start>0)].drop_duplicates()
    windows=windows.loc[(windows.chrom!='chrX')&(windows.chrom!='chrY')&(windows.chrom!='chrM'),:]
    startInds=binsDf.reset_index().merge(windows[['chrom','start']],how='inner',on=['chrom','start'])['index']
    endInds=binsDf.reset_index().merge(windows[['chrom','end']],how='inner',on=['chrom','end'])['index']
    indDf=pd.DataFrame({'startInd':startInds,'endInd':endInds})
    return ((indDf.startInd+indDf.endInd)//2).values

def plotCorrMatrix(df, corrMethod='pearson', saveName=None,title=None):
    correlationsDf=df.fillna(0).corr(method=corrMethod)
    # Generate a mask for the upper triangle
    mask = np.triu(np.ones_like(correlationsDf, dtype=np.bool))

    # Set up the matplotlib figure
    f, ax = plt.subplots(figsize=(5+correlationsDf.shape[1], 1+correlationsDf.shape[1]))
    # Generate a custom diverging colormap
#     cmap = sns.diverging_palette(220, 0, as_cmap=True)

    # Draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(correlationsDf, mask=mask, cmap="vlag", vmax=.3, center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5,"label": corrMethod+' correlation'})
    if title is not None:
        plt.title(title)
    if saveName is not None:
        plt.savefig(saveName)
    return correlationsDf

In [ ]:
def OLSBackwardSelectStep(X, y, selected, sigThreshold):
    pVals=sm.OLS(y,X[:, selected]).fit().pvalues
    if pVals.max()>sigThreshold:
        selected[np.argwhere(selected)[np.argmax(pVals)]]=False
        converged=False
    else:
        converged=True
    return selected, converged
    
def OLSForwardSelectStep(X, y, selected, sigThreshold):
    minMaxPVal=1
    for ind in np.argwhere(~selected):
        newSelectedCandidate=np.copy(selected)
        newSelectedCandidate[ind]=True
        maxPVal=sm.OLS(y,X[:,newSelectedCandidate]).fit().pvalues.max()
        if maxPVal<minMaxPVal:
            newSelected=newSelectedCandidate
            minMaxPVal=maxPVal
    if minMaxPVal>sigThreshold:
        converged=True
    else:
        selected=newSelected
        converged=False
    return selected, converged

def OLSBackwardSelect(X, y, sigThreshold):
    converged=False
    selected=np.ones(X.shape[1]).astype('bool')
    while (np.sum(selected)>1) & (not converged):
        selected, converged = OLSBackwardSelectStep(X, y, selected=selected, sigThreshold=sigThreshold)
    return selected

def OLSForwardSelect(X, y, sigThreshold):
    converged=False
    selected=np.zeros(X.shape[1]).astype('bool')
    selected[0]=True
    while (np.sum(selected)<X.shape[1]) & (not converged):
        selected, converged = OLSForwardSelectStep(X, y, selected=selected, sigThreshold=sigThreshold)
    return selected

In [ ]:
peakFiles=glob.glob('../data/peakBeds/*')
peakIndsDict=dict(zip([os.path.basename(file).split('.')[0] for file in peakFiles],[getPeakInds(file) for file in peakFiles]))
saveDir='../outputs/'

In [ ]:
predictorList=['ES_cisTotalRatio',
            'zygo_cisTotalRatio',
            'pachy_cisTotalRatio',
            'ES_E1chunks',
            'zygo_E1chunks',
            'pachy_E1chunks',
            'ES_fireScore',
            'zygo_fireScore',
            'pachy_fireScore',
            'ES_logInsScore',
            'zygo_logInsScore',
            'pachy_logInsScore',
            'chromHMMstate_1_binned',
            'chromHMMstate_2_binned',
            'chromHMMstate_3_binned',
            'chromHMMstate_4_binned',
            'chromHMMstate_5_binned',
            'chromHMMstate_6_binned',
            'chromHMMstate_7_binned',
            'vara2019_PDrad21l',
            'vara2019_PDctcf',
            'margolin2014RNAPII16dpp',
            'Nitzsche2011_ESC_RAD21',
            'Nitzsche2011_ESC_CTCF',
            'ES_RNAP2_binned_maxScore',
]
simpleVarNames=['ES cis-total ratio',
                'Zygonema cis-total ratio',
                'Pachynema cis-total ratio',
                'ES compartment vector',
                'Zygonema compartment vector',
                'Pachynema compartment vector',
                'ES FIRE score',
                'Zygonema FIRE score',
                'Pachynema FIRE score',
                'ES log insulation score',
                'Zygonema log insulation score',
                'Pachynema log insulation score',
                'H3K4me3 chromatin',
                'H3K4me1/3 chromatin',
                'H3K4me1 chromatin',
                'H3K4me1+K36me3 chromatin',
                'H3K36me3 chromatin',
                'unmarked chromatin',
                'H3K27me3 chromatin',
                'Meiotic RAD21L ChIP-seq',
                'Meiotic CTCF ChIP-seq',
                'Meiotic RNAPII ChIP-seq',
                'ES RAD21 ChIP-seq',
                'ES CTCF ChIP-seq',
                'ES RNAPII ChIP-seq']

In [ ]:
hotspotLocs=list(set().union(peakIndsDict['smagulova2016_B6xCAST_DSB_fraglen1000_peaks'],peakIndsDict['Baker2015_prdm9_B6xCAST_peaks']))
hotspotDf=signalDf.iloc[hotspotLocs,:]
DSBdf=signalDf.iloc[peakIndsDict['smagulova2016_B6xCAST_DSB_fraglen1000_peaks'],:]
prdm9Df=signalDf.iloc[peakIndsDict['Baker2015_prdm9_B6xCAST_peaks'],:]

In [ ]:
avgPredictorList=[predictor+'_roll101mean' for predictor in predictorList]
avgSimpleVarNames=[simpleVarName+' 500kb avg' for simpleVarName in simpleVarNames]
fullPredictorList=predictorList+avgPredictorList
fullSimpleVarNames=simpleVarNames+avgSimpleVarNames
simpleDict=dict(zip(fullPredictorList,fullSimpleVarNames))
correlationDf=hotspotDf.loc[:,fullPredictorList]
correlationDf.rename(simpleDict, axis=1, inplace=True)
plotCorrMatrix(correlationDf, corrMethod='spearman', saveName=f'{saveDir}/correlations.pdf')


In [ ]:
predictorList2=['Baker2015_prdm9_B6xCAST',
               'smagulova2016_B6xCAST_DSB_fraglen1000',
               'yin2019_crossoversCast1C_binned_weightedScore_medianNormalized',
            'chromHMMstate_5_binned',]
simpleVarNames2=['PRDM9 ChIP-seq',
                'DMC1-SSDS ChIP-seq',
                'Crossover score',
                'H3K36me3 chromatin',]
simpleDict2=dict(zip(predictorList2,simpleVarNames2))

correlationDf2=hotspotDf.loc[:,predictorList2]
correlationDf2.rename(simpleDict2, axis=1, inplace=True)
plotCorrMatrix(correlationDf2, corrMethod='spearman', saveName=f'{saveDir}/recombH3K36me3corrsHotspotsOnly.pdf')

correlationDf3=signalDf.loc[:,predictorList2]
correlationDf3.rename(simpleDict2, axis=1, inplace=True)
plotCorrMatrix(correlationDf3, corrMethod='spearman', saveName=f'{saveDir}/recombH3K36me3corrsGenomewide.pdf')

In [ ]:
predictors=' + '.join(fullPredictorList)
output='yin2019_crossoversCast1C_binned_weightedScore_medianNormalized'
formula=f'{output} ~ {predictors}'

y, X = dmatrices(formula, data=hotspotDf, return_type='matrix')

numVars=X.shape[1]-1

scaler = StandardScaler()
pca=PCA(n_components=numVars)
varIndex=X.design_info.column_names[1:]
PCindex=[f'PC{ind}' for ind in range(1,numVars+1)]
X2=np.concatenate((X[:,0,None], pca.fit_transform(scaler.fit_transform(X[:,1:]))), axis=1)
plt.rcParams.update({'font.size': 16})

fig=plt.figure(figsize=(6,0.25*numVars))
ax1=sns.barplot(y=PCindex, x=pca.explained_variance_, palette=[[0.5,0.5,0.5]])
ax1.set_title('PCA percent explained variance')
plt.savefig(f'{saveDir}/PCAexplainedVariance.pdf')
print(pca.explained_variance_)
plt.rcParams.update({'font.size': 24})


In [ ]:
components=pd.DataFrame(pca.transform(np.identity(numVars)), index=varIndex, columns=PCindex)
components.rename(simpleDict, axis=0, inplace=True)

componentsDf=components.unstack().reset_index()
componentsDf['variable']=[varName.rsplit(' 500kb',1)[0] for varName in componentsDf.level_1]
componentsDf['window']=['500kb avg' if '500kb' in varName else 'local 5kb bin' for varName in componentsDf.level_1]
componentsDf.rename({0:'score','level_0':'PCind'}, axis=1, inplace=True)
g=sns.catplot(y='score', x='variable', hue='window', col='PCind',col_wrap=5,kind='bar', data=componentsDf, 
               height=2, aspect=3, sharex=True, sharey=True)
for ax in g.axes.ravel():
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
plt.tight_layout()
plt.savefig(f'{saveDir}/PCAallLoadings.pdf')

In [ ]:
components=pd.DataFrame(pca.transform(np.identity(numVars)), index=varIndex, columns=PCindex)
components['varType']=[varName.rsplit('_roll',1)[0] for varName in components.index.tolist()]
rollVarBool=[len(varName.rsplit('_roll',1)) for varName in components.index.tolist()]
components['rollVar']=['500kb average' if rv>1.5 else 'local 5kb bin' for rv in rollVarBool]

palettes=[[(1,0.5,0),(1,0.75,0.5)],
        [(1,0,0),(1,0.5,0.5)],
         [(0,0,1),(0.7,0.7,1)],
         [(0.4,0.2,0.4),(0.7,0.4,0.7)]]

palettes=[[(1,0.5,0),(1,0.75,0.5)],
        [(1,0,0),(1,0.5,0.5)],
         [(0,0,1),(0.7,0.7,1)],
         [(0.4,0.2,0.4),(0.7,0.4,0.7)],
         [(0.3,0.3,0.3),(0.6,0.6,0.6)]]

for ind,PCname in enumerate(PCindex[:5]):
    plt.figure(figsize=(15,2))
    ax=sns.barplot(y=components[PCname], x=components.varType, hue=components.rollVar, palette=palettes[ind])
    ax.set_xticklabels(simpleVarNames)
    ax.set_xlabel("")
    ax.set_ylabel(PCname+" loadings")
    plt.xticks(rotation=90)
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.savefig(f'{saveDir}/{PCname}.pdf')

In [ ]:
predictors=' + '.join(['start','smagulova2016_B6xCAST_DSB_fraglen1000'])
output='yin2019_crossoversCast1C_binned_weightedScore_medianNormalized'
formula=f'{output} ~ {predictors}'

y, X = dmatrices(formula, data=DSBdf, return_type='matrix')
model=sm.OLS(y,X)
results=model.fit()
print(results.summary())

In [ ]:
predictors=' + '.join(['start','smagulova2016_B6xCAST_DSB_fraglen1000']+fullPredictorList)
output='yin2019_crossoversCast1C_binned_weightedScore_medianNormalized'
formula=f'{output} ~ {predictors}'

y, X = dmatrices(formula, data=DSBdf, return_type='matrix')

X2=np.concatenate((X[:,0,None], scaler.fit_transform(X[:,1:3]), pca.transform(scaler.fit_transform(X[:,3:]))), axis=1)

model=sm.OLS(y,X2)
results=model.fit()
print(results.summary())
plt.figure(figsize=(6,0.25*len(fullPredictorList)))
barplot=sns.barplot(y=['start','smagulova2016_B6xCAST_DSB_fraglen1000']+PCindex, x=results.tvalues[1:])
if (type(model).__name__=='Logit'):
    threshold=2.58
elif (type(model).__name__=='OLS'):
    threshold=2.33
plt.axvline(threshold, color='k', linewidth=1, linestyle='--')
plt.axvline(-threshold, color='k', linewidth=1, linestyle='--')

In [ ]:
sigThreshold=0.001/X2.shape[1] #bonferroni
varNames=['Intercept','start','smagulova2016_B6xCAST_DSB_fraglen1000'] + PCindex

backwardSelected=OLSBackwardSelect(X2, y, sigThreshold)
backwardSelectedPCnames=[varName for idx,varName in enumerate(varNames) if backwardSelected[idx]]

forwardSelected=OLSForwardSelect(X2, y, sigThreshold)
forwardSelectedPCnames=[varName for idx,varName in enumerate(varNames) if forwardSelected[idx]]
print(forwardSelectedPCnames,backwardSelectedPCnames)


In [ ]:
model=sm.OLS(y,X2[:, forwardSelected])
results=model.fit()

plt.figure(figsize=(6,0.35*sum(forwardSelected)))
palette=2*[(0.6,0.6,0.6)]+[(1,0.5,0),
         (1,0,0),
         (0,0,1),
         (0.4,0.2,0.4)
        ]+len(forwardSelected)*[(0.6,0.6,0.6)]
ax=sns.barplot(y=forwardSelectedPCnames[1:], x=results.tvalues[1:], palette=palette)
simpleAxisLabels=['Chromosomal position', 'DMC1 ChIP-seq score'] + [ticklabel.get_text() for ticklabel in ax.get_yticklabels()[2:]]
ax.set_yticklabels(simpleAxisLabels)
ax.set_xlabel('t-statistic')
threshold=stats.t.ppf(sigThreshold/2, df=X2.shape[0]-X2.shape[1]-1)
plt.axvline(threshold, color='k', linewidth=1, linestyle='--', label='Bonferroni-corrected \np<0.001 threshold')
plt.axvline(-threshold, color='k', linewidth=1, linestyle='--')
ax.legend()
plt.savefig(f'{saveDir}/CO_results.pdf')
print(results.summary())

In [ ]:
predictors=' + '.join(['Baker2015_prdm9_B6xCAST'])
output='smagulova2016_B6xCAST_DSB_fraglen1000'
formula=f'{output} ~ {predictors}'

y, X = dmatrices(formula, data=prdm9Df, return_type='matrix')

model=sm.OLS(y,X)
results=model.fit()
print(results.summary())

In [ ]:
predictors=' + '.join(['start','Baker2015_prdm9_B6xCAST']+fullPredictorList)
output='smagulova2016_B6xCAST_DSB_fraglen1000'
formula=f'{output} ~ {predictors}'

y, X = dmatrices(formula, data=prdm9Df, return_type='matrix')

X2=np.concatenate((X[:,0,None], scaler.fit_transform(X[:,1:3]), pca.transform(scaler.fit_transform(X[:,3:]))), axis=1)

model=sm.OLS(y,X2)
results=model.fit()
plt.figure(figsize=(6,0.25*len(fullPredictorList)))
barplot=sns.barplot(y=['start','grey2017_prdm9CAST_binned_maxScore']+PCindex, x=results.tvalues[1:])
if (type(model).__name__=='Logit'):
    threshold=2.58
elif (type(model).__name__=='OLS'):
    threshold=2.33
plt.axvline(threshold, color='k', linewidth=1, linestyle='--')
plt.axvline(-threshold, color='k', linewidth=1, linestyle='--')

In [ ]:
sigThreshold=0.001/X2.shape[1] #bonferroni
varNames=['Intercept','start','Baker2015_prdm9_B6xCAST'] + PCindex

backwardSelected=OLSBackwardSelect(X2, y, sigThreshold)
backwardSelectedPCnames=[varName for idx,varName in enumerate(varNames) if backwardSelected[idx]]

forwardSelected=OLSForwardSelect(X2, y, sigThreshold)
forwardSelectedPCnames=[varName for idx,varName in enumerate(varNames) if forwardSelected[idx]]
print(forwardSelectedPCnames,backwardSelectedPCnames)


In [ ]:
model=sm.OLS(y,X2[:, forwardSelected])
results=model.fit()
plt.figure(figsize=(6,0.35*sum(forwardSelected)))
palette=1*[(0.6,0.6,0.6),(0.6,0.6,0.6)]+[(1,0.5,0)]+len(forwardSelected)*[(0.6,0.6,0.6)]
ax=sns.barplot(y=forwardSelectedPCnames[1:], x=results.tvalues[1:], palette=palette)
simpleAxisLabels=['Chromosomal position','PRDM9 ChIP-seq score'] + [ticklabel.get_text() for ticklabel in ax.get_yticklabels()[2:]]
ax.set_yticklabels(simpleAxisLabels)
ax.set_xlabel('t-statistic')
threshold=stats.t.ppf(sigThreshold/2, df=X2.shape[0]-X2.shape[1]-1)
plt.axvline(threshold, color='k', linewidth=1, linestyle='--', label='Bonferroni-corrected \np<0.001 threshold')
plt.axvline(-threshold, color='k', linewidth=1, linestyle='--')
ax.legend()
plt.savefig(f'{saveDir}/DSB_results.pdf')
print(results.summary())

In [ ]:
predictors=' + '.join(['start','Baker2015_prdm9_B6xCAST']+fullPredictorList)
output='yin2019_crossoversCast1C_binned_weightedScore_medianNormalized'
formula=f'{output} ~ {predictors}'

y, X = dmatrices(formula, data=prdm9Df, return_type='matrix')

X2=np.concatenate((X[:,0,None], scaler.fit_transform(X[:,1:3]), pca.transform(scaler.fit_transform(X[:,3:]))), axis=1)

model=sm.OLS(y,X2)
results=model.fit()
plt.figure(figsize=(6,0.25*len(fullPredictorList)))
barplot=sns.barplot(y=['start','Baker2015_prdm9_B6xCAST']+PCindex, x=results.tvalues[1:])
if (type(model).__name__=='Logit'):
    threshold=2.58
elif (type(model).__name__=='OLS'):
    threshold=2.33
plt.axvline(threshold, color='k', linewidth=1, linestyle='--')
plt.axvline(-threshold, color='k', linewidth=1, linestyle='--')

In [ ]:
sigThreshold=0.001/X2.shape[1] #bonferroni
varNames=['Intercept','start','Baker2015_prdm9_B6xCAST'] + PCindex

backwardSelected=OLSBackwardSelect(X2, y, sigThreshold)
backwardSelectedPCnames=[varName for idx,varName in enumerate(varNames) if backwardSelected[idx]]

forwardSelected=OLSForwardSelect(X2, y, sigThreshold)
forwardSelectedPCnames=[varName for idx,varName in enumerate(varNames) if forwardSelected[idx]]
print(forwardSelectedPCnames,backwardSelectedPCnames)


In [ ]:
model=sm.OLS(y,X2[:, forwardSelected])
results=model.fit()
plt.figure(figsize=(6,0.35*sum(forwardSelected)))
palette=2*[(0.6,0.6,0.6)]+[(1,0,0),
         (0,0,1),
         (0.4,0.2,0.4)
        ]+len(forwardSelected)*[(0.6,0.6,0.6)]
ax=sns.barplot(y=forwardSelectedPCnames[1:], x=results.tvalues[1:], palette=palette)
simpleAxisLabels=['Chromosomal position','PRDM9 ChIP-seq score'] + [ticklabel.get_text() for ticklabel in ax.get_yticklabels()[2:]]
ax.set_yticklabels(simpleAxisLabels)
ax.set_xlabel('t-statistic')
threshold=stats.t.ppf(sigThreshold/2, df=X2.shape[0]-X2.shape[1]-1)
plt.axvline(threshold, color='k', linewidth=1, linestyle='--', label='Bonferroni-corrected \np<0.001 threshold')
plt.axvline(-threshold, color='k', linewidth=1, linestyle='--')
ax.legend()
plt.savefig(f'{saveDir}/PRDM9toCO_results.pdf')
print(results.summary())